In [375]:
from proxy_chrome_browser import get_chromedriver, build_proxy_extension
import moviepy.editor as mpy
import glob
import os
from time import sleep
from tqdm import tqdm

In [376]:
br = get_chromedriver(headless=True,path='/home/miko/Downloads/chromedriver-v-106-0-5249-61')

In [377]:
def find_elm(tag,driver=br,_all=False,_by="css"):
    from selenium.webdriver.common.by import By
    if _by=="css":
        _by=By.CSS_SELECTOR
    elif _by=="link_text":
        _by=By.PARTIAL_LINK_TEXT
    elif _by=="link_text_exact":
        _by=By.LINK_TEXT
        
    try:
        if _all:
            out = driver.find_elements(by=_by,value=tag)
        else:
            out = driver.find_element(by=_by,value=tag)
    except:
        #print('None')
        out = None
    return out

In [378]:
def save_screenshot(elm,filename):
    assert filename[-3:] == "png", "filename should end with png"
    with open(filename,"wb") as f:
        f.write(elm.screenshot_as_png)

In [379]:
def scroll_into_view(elm):
     #source: -https://stackoverflow.com/questions/25733877/selenium-how-do-you-check-scroll-position
     #loc = elm.location
     #current_view_script = """ var doc = document.documentElement;
     #var left = (window.pageXOffset || doc.scrollLeft) - (doc.clientLeft || 0);
     #var top = (window.pageYOffset || doc.scrollTop)  - (doc.clientTop || 0);
     #taking screenshot will scroll into view. IF this stops working can use javascript above
     elm.screenshot_as_png
     sleep(.5)


In [380]:
post_url = "https://www.reddit.com/r/AskReddit/comments/yiazab/would_you_support_a_mandatory_retirement_age_of/"

In [381]:
br.get(post_url)

In [345]:
#br.switch_to.active_element.text

In [384]:
# expand replies
num_replies_threshold = 90
get_replies = True
num_total_loops = 5
current_loop_num = 0
clicked_links = []
while get_replies:
    more_replies_divs = find_elm("div[id^='moreComments']",_all=True)
    expandable_divs = []
    for div in more_replies_divs:
        more_replies_link = find_elm("p",driver=div)
        #p = br.execute_script("return arguments[0].parentNode;",more_replies_link)
        if not more_replies_link:
            continue
        if more_replies_link.text == '':
            continue
        num_replies = int(more_replies_link.text.split()[0].replace(",",""))
        if num_replies >= num_replies_threshold:
            if more_replies_link not in clicked_links:
                expandable_divs.append(more_replies_link)
            else:
                continue
    for div in expandable_divs:
        #link_elm = find_elm("div[data-testid^='search-scope-pill-text']")
        try:
            scroll_into_view(more_replies_link)
            div.click()
            sleep(1)
        except:
            continue  
    if len(expandable_divs) == 0:
        get_replies = False
    if current_loop_num >= num_total_loops:
        get_replies = False
    else:
        current_loop_num += 1
    print("getting more replies")
    #todo: find a better stopping condition

getting more replies
getting more replies
getting more replies
getting more replies
getting more replies
getting more replies


In [385]:
# get comments
comments = find_elm("div[class*='Comment']",_all=True)

In [386]:
len(comments)

497

In [387]:
submission_title = br.current_url.split("/")[-2]
asset_dir="assets/demo_png_2_vid"
thread_ids = []

In [388]:
def save_comment_text(filename,comment):
    comment_text = find_elm("p",driver=comment).text
    with open(filename,"w") as f:
        f.write(comment_text)

In [389]:
#save threaded comment screenshot as png to directory
vote_count_threshold = 100

for comment in tqdm(comments):
    # filter deleted comments
    if "Comment deleted by user" in comment.text:
        continue
    
    # some comments arnt expanded, skipping
    if len(comment.text.split("\n")) <= 4:
           continue
           
    #filter highly voted comments
    vote_div = find_elm("div[id^='vote-arrows']",driver=comment)
    multiply_1k = 'k' in vote_div.text
    if multiply_1k:
        vote_count = float(vote_div.text.replace("k","")) * 1000
    else:
        vote_count = int(vote_div.text)
    
    comment_lvl = find_elm("span",comment).text
    if "1" in comment_lvl:
        user = comment.text.split("\n")[1]
        vote_count_txt = vote_div.text.replace(".","_")
        thread_id = f"{user}-{vote_count_txt}"
        
        comment_id=0
    else:
        comment_id+=1
    
    save_comment = False
    if vote_count >= vote_count_threshold:
        save_comment = True
    
    if save_comment:
        
        thread_ids.append(thread_id)
        folder = f"{asset_dir}/{submission_title}/{thread_id}"
        if not os.path.exists(folder):
            os.makedirs(folder)
        save_screenshot(comment,f"{folder}/{comment_id:04}.png")
        save_comment_text(filename=f"{folder}/{comment_id:04}.txt",comment=comment)



  0%|                                                                                                                                                                                                                                                     | 0/497 [00:00<?, ?it/s]

  0%|▍                                                                                                                                                                                                                                            | 1/497 [00:00<06:52,  1.20it/s]

  0%|▉                                                                                                                                                                                                                                            | 2/497 [00:02<08:51,  1.07s/it]

  1%|█▍                                                                                                                                                                   

In [370]:
def build_thread_vid(thread_id):
    images = glob.glob(f"{asset_dir}/{submission_title}/{thread_id}/*.png")
    # https://zulko.github.io/moviepy/examples/examples.html
    clips = []
    
    for i,image in enumerate(images):
        clip = mpy.ImageClip(image).set_duration(10).set_start(i*10)
        #clip = clip.set_start(5) # TODO: change duration based on audio
        #clip = clip.resize((1080,1920)) #TODO add tts audio
        clips.append(clip)
    
    final_clip = mpy.CompositeVideoClip(clips)
    final_clip.write_videofile(f"{asset_dir}/{submission_title}/{thread_id}/thread_clip.mp4",fps=30,logger=None)
    

In [374]:
comments[0].text

MaxRetryError: HTTPConnectionPool(host='localhost', port=33467): Max retries exceeded with url: /session/97a7914d750d50352d2cb357d4eabf04/element/f79fcd2c-2e8a-47aa-a317-8ca7fc5814a4/text (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f1984c9e910>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [371]:
# save threads to clips
# vote_div.text
thread_ids = list(set(thread_ids))
for thread in tqdm(thread_ids):
    build_thread_vid(thread)



  0%|                                                                                                                                                                                                                                                      | 0/16 [00:00<?, ?it/s]

  6%|██████████████▉                                                                                                                                                                                                                               | 1/16 [00:04<01:11,  4.80s/it]

 12%|█████████████████████████████▊                                                                                                                                                                                                                | 2/16 [00:23<03:01, 12.95s/it]

 19%|████████████████████████████████████████████▋                                                                                                                        

In [ ]:
# save c to 

In [254]:
comments[0].text.split("\n")[9]

'18.2k'

In [311]:
images = glob.glob(f"{asset_dir}/{thread_ids[0]}/{}/*")

In [316]:
glob.glob()

['assets/demo_png_2_vid/would_you_support_a_mandatory_retirement_age_of/mattjf22-18_2k/0.png',
 'assets/demo_png_2_vid/would_you_support_a_mandatory_retirement_age_of/mattjf22-18_2k/1.png']

In [329]:
mpy.concatenate_videoclips?

Signature:
mpy.concatenate_videoclips(
    clips,
    method='chain',
    transition=None,
    bg_color=None,
    ismask=False,
    padding=0,
)
Docstring:
Concatenates several video clips

Returns a video clip made by clip by concatenating several video clips.
(Concatenated means that they will be played one after another).

There are two methods:

- method="chain": will produce a clip that simply outputs
  the frames of the succesive clips, without any correction if they are
  not of the same size of anything. If none of the clips have masks the
  resulting clip has no mask, else the mask is a concatenation of masks
  (using completely opaque for clips that don't have masks, obviously).
  If you have clips of different size and you want to write directly the
  result of the concatenation to a file, use the method "compose" instead.

- method="compose", if the clips do not have the same
  resolution, the final resolution will be such that no clip has
   to be resized.
   As a conseque